# Word2Vecを用いた類語の取得
<!--
[学習済みのword2vecのモデル](https://github.com/Kyubyong/wordvectors)

[学習済みのword2vecのモデルのロード](https://blog.amedama.jp/entry/gensim-fasttext-pre-trained-word-vectors)
-->
[データ](https://fasttext.cc/docs/en/crawl-vectors.html)

In [ ]:
#!curl https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz --output cc.ja.300.vec.gz 

### 出力用のディレクトリを作成する

In [ ]:
import os 
out_dir='./out'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

### 読み込みに時間がかかる(10分程度)

- pickle でダンプしておく

In [ ]:
%%time
import gensim
binary=True
data_path='wiki.ja.vec'; binary=False
data_path='cc.ja.300.vec.gz'; binary=False
model = gensim.models.KeyedVectors.load_word2vec_format(data_path, binary=binary,
                                                        unicode_errors='ignore')

import pickle as pkl
out_path=os.path.join(out_dir,data_path+'.pkl')
with open(out_path,'wb') as fw:
    pkl.dump(model,fw)

# 事前保存のデータを読み込み(直接読むより早い)

In [ ]:
%%time
import pickle as pkl
data_path='wiki.ja.vecl'
data_path='cc.ja.300.vec.gz'
in_path=os.path.join(out_dir,data_path+'.pkl')
with open(in_path,'rb') as fr:
    model0=pkl.load(fr)

In [ ]:
type(model0)

In [ ]:
print(list(model0.index_to_key)[:10])
print(len(model0.index_to_key))


In [ ]:
#model0.key_to_index

In [ ]:
model0['猫'].shape

In [ ]:
# import gensim
# model0 = gensim.models.Word2Vec.load('./ja/ja.bin')

In [ ]:

def get_synonyms_w2v(text,model):
    results = []
    #for word, sim in model.wv.most_similar(text, topn=10):
    for word, sim in model.most_similar(text, topn=10):
        results.append({'term': word, 'similarity': sim})
    return results


def calc_similarity_w2v(text1, text2,model):
    sim = model.wv.similarity(text1, text2)
    return sim




In [ ]:
probe_words=['フランス','東京','ニューヨーク',
             #'自由の女神',
             'ウルトラマン',
             '中学校',
             '政府',
             #'6月',
             #'テレビドラマ',
             'ピカソ',
             '夏',
             'WTO',
             'SARS',
            ]

for probe_word in probe_words:
    word_similar=get_synonyms_w2v(probe_word,model0)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{probe_word} 〜　\n{words}\n')

    
    

# アナロジー　（意味の加減算)

In [ ]:
def analogy(X_Y, x,model):
    X, Y = X_Y
    results = []
    #for word, sim in model.wv.most_similar(positive=[Y, x], negative=[X], topn=10):
    for word, sim in model.most_similar(positive=[Y, x], negative=[X], topn=10):
        results.append({'term': word, 'similarity': sim})
    return results

In [ ]:
X_Y_x_list=[( 'フランス','パリ','ドイツ'),  #首都
            ('イギリス','ロンドン','日本'),  #首都
            ('日本','東京','ドイツ'),  #首都
            ('少年','少女','夫'),  #性別
            ('日本','東京', '韓国' ),  # NG 韓国がボキャブラリー辞書にない
            ('日本','東京', '中国'),   # NG　中国がボキャブラリー辞書にない
            ('パリ', 'エッフェル塔','東京'), # 観光資源
           ]

for X,Y, x in X_Y_x_list:
    word_similar=analogy((X,Y),x,model0)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{Y} - {X} + {x} 〜　\n{words}\n')



# 日本語Wikipedia 全ページの文章でWord2Vecを学習

In [ ]:
import re
import unicodedata
from bs4 import BeautifulSoup
#import lec05_parser as parser
import MeCab
from glob import glob
import os
from tqdm.notebook import tqdm

# クレンジング：正規化と空白の圧縮

In [ ]:
translation_table=str.maketrans(dict(zip('()!', '（）！')))  # 半角 -> 全角  
#
#  クレンジング：　正規化と，　空白の圧縮
#
def cleanse(text,debug=False):
    if debug:
        return text
    text=unicodedata.normalize('NFKC', text).translate(translation_table)  #  テキストの正規化
    text=re.sub(r'\s+', ' ',text)  # 空白の正規化 半角の空白1つに。
    #text=cleanse_post(text)
    return text

def insert_nl_at_eos_char(text):  # insert newline's
    text=re.sub('([。])', '\\1\n',text)
    return text

def cleanse_ext(text):  #  for word2vec special
    text=re.sub('Section::::','',text)
    text=re.sub('[！「」『』（），、]','',text)
    return text
                

# 文単位で分かち書きされたファイルの作成

In [ ]:
# コーパス作り
WIKIDUMP_DIR='./wikipedia_text'
eos_mark='<EOS>'

def get_each_contetns_by_text(wikidir):
    for file in tqdm(glob(os.path.join(wikidir,'*/wiki_*'))):
        #print(file)
        with open(file,'r') as f:
            read_text=f.read()
            soup=BeautifulSoup(read_text)
            for doc in soup.find_all(['doc']):
                text=''
                input_text=doc.text
                input_text=cleanse(input_text)
                input_text=insert_nl_at_eos_char(input_text)
                for line in input_text.split('\n'):
                    line=line.strip()
                    if len(line)>0:
                        #print(f'**{line}**')
                        text += cleanse_ext(line)
                        if line[-1] in ['。','！']:
                            text=text[:-1]+eos_mark  # remove ['。','！'] but insert eos
                        else:
                            text+=eos_mark
                            #
                            pass
                        text += '\n'
                title=doc['title']
                yield cleanse(text), cleanse(title)
                #yield text, title
    
def convert_and_concat_wakati(out_dir='.'):
    with open(os.path.join(out_dir,'wiki_wakati_nl_Q.txt'),'w') as fw,\
        open(os.path.join(out_dir,'wiki_title_Q.txt'),'w') as f_title:
        count=0    
        #mecab=parser.MeCab('-Owakati')
        mecab= MeCab.Tagger('-Owakati')
        for text, title in  get_each_contetns_by_text(WIKIDUMP_DIR):
            print('　'*60,end='\r')
            print(f'{count}:{title[:45]}',end='\r')
            f_title.write(f'{count}:{title}\n')
            #print(mecab.parse(text))
            wakati_words=mecab.parse(text)
            #wakati_words=wakati.parse(text)   # こっちは エッフェル塔 が分解される
            wakati_words=repair_eos(wakati_words)
            fw.write(wakati_words)
            #print(text)
            if count>1:
                #break
                pass
            count += 1
            #if count>5: break  # for debug
        print('done')
        del mecab
        
def repair_eos(text):
    text=re.sub('< EOS > ','\n', text)
    return text
    
# def repair_eos(text):
#     text = '<SOS> '+text
#     text=re.sub('< EOS >','<EOS>\n<SOS>', text)
#     text=re.sub('<SOS>$','',text)
#     #print(f'* {text} *')
#     return text
    

# 分かち書き処理の実行

In [ ]:
%%time 
convert_and_concat_wakati(out_dir)

#  実行の記録
2020/11/11 <br>
```
見出し数：1,179,168
分かち書き処理：
CPU times: user 20min 54s, sys: 3min 18s, total: 24min 12s
Wall time: 1h 18min 2s
```

2021/11/18<br>
```
done101:福生市立福生第五小学校　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
CPU times: user 21min 48s, sys: 1min 49s, total: 23min 38s
Wall time: 24min 26s
```

# gensimのword2vec で
# 日本語wikipedia全コンテンツを学習

In [ ]:
from gensim.models import word2vec
vector_size=100
vector_size=300
%time data = word2vec.Text8Corpus(os.path.join(out_dir,'wiki_wakati_nl_Q.txt'))
%time model =  word2vec.Word2Vec(data, vector_size=vector_size)  # 100次元で学習
#
out_fname=f"wiki_{vector_size}.model"
out_path=os.path.join(out_dir,out_fname)
print(f'saving {out_path} ... ')
%time model1.save(out_path)  
print("ok") 

In [ ]:

out_fname=f"wiki_{vector_size}.model"
out_path=os.path.join(out_dir,out_fname)
print(f'saving {out_path} ... ')
%time model1.save(out_path)  
print("ok") 

#  実行の記録
2020/11/09 <br>
```
CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.06 µs
CPU times: user 1h 34min 30s, sys: 1min 7s, total: 1h 35min 38s
Wall time: 36min 14s
CPU times: user 2.07 s, sys: 1.13 s, total: 3.2 s
Wall time: 3.9 s
ok
```


```
ファイルサイズ：
wiki_wakati_nl.txt                 3,147,077,302 バイト（ディスク上の3.15 GB）
wiki.model                         67,779,585 バイト（ディスク上の70.8 MB）
wiki.model.trainables.syn1neg.npy  373,465,328 バイト（ディスク上の376.9 MB）
wiki.model.wv.vectors.npy          373,465,328 バイト（ディスク上の375.7 MB）
```

2021.11.18<br>
```
vector_size=100
CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.9 µs
CPU times: user 1h 17min 40s, sys: 41.5 s, total: 1h 18min 21s
Wall time: 27min 34s
CPU times: user 210 ms, sys: 920 ms, total: 1.13 s
Wall time: 1.19 s
ok
```

生成ファイル<br>
```
15787032  11 18 14:19 wiki.model
177484928 11 18 14:19 wiki.model.syn1neg.npy
177484928 11 18 14:19 wiki.model.wv.vectors.npy
```

```
vector_size=300
CPU times: user 8 µs, sys: 20 µs, total: 28 µs
Wall time: 31 µs
CPU times: user 1h 48minCPU times: user 8 µs, sys: 20 µs, total: 28 µs
Wall time: 31 µs
CPU times: user 1h 48min
saving wiki_300.model ... 
CPU times: user 249 ms, sys: 2.2 s, total: 2.45 s
Wall time: 2.67 s
oksaving wiki_300.model ... 
CPU times: user 249 ms, sys: 2.2 s, total: 2.45 s
Wall time: 2.67 s
ok

```

# 学習した Word2Vec の利用 (モデルロード）

In [ ]:
%%time
from gensim.models import word2vec
import gensim
vector_size=100
vector_size=300

in_fname=f"wiki_{vector_size}.model"
in_path=os.path.join(out_dir,in_fname)
print(f'loading {in_path} ... ')
model1 = gensim.models.Word2Vec.load(in_path) 

In [ ]:
type(model1)

# 同義語

In [ ]:
probe_words=['フランス','東京','ニューヨーク',
             #'自由の女神','ウルトラマン',
             #'中学校',
             '政府',
             #'6月',
             #'テレビドラマ',
             'ピカソ','夏','WTO','SARS','コロナ']
for probe_word in probe_words:
    word_similar=get_synonyms_w2v(probe_word,model1.wv)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{probe_word} 〜　\n{words}\n')

    

# アナロジー　（意味の加減算)

In [ ]:
vector_size=100
in_fname=f"wiki_{vector_size}.model"
in_path=os.path.join(out_dir,in_fname)
print(f'loading {in_path} ... ')
model100 = gensim.models.Word2Vec.load(in_path) 

In [ ]:
X_Y_x_list=[( 'フランス','パリ','ドイツ'),  #首都
            ('イギリス','ロンドン','日本'),  #首都
            ('日本','東京','ドイツ'),  #首都
            ('少年','少女','夫'),  #性別
            ('日本','東京', '韓国' ),  # NG 韓国がボキャブラリー辞書にない
            ('日本','東京', '中国'),   # NG　中国がボキャブラリー辞書にない
            #('パリ', 'エッフェル塔','東京'), # 観光資源
           ]

for X,Y, x in X_Y_x_list:
    print('model_300')
    print('*'*30)
    word_similar=analogy((X,Y),x,model1.wv)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{Y} - {X} + {x} 〜　\n{words}\n')
    #----
    print('model_100')
    word_similar=analogy((X,Y),x,model100.wv)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{Y} - {X} + {x} 〜　\n{words}\n')




# ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ <br>

# ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ <br>


# ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ <br>


